In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os

In [2]:
import keras_tuner as kt
from keras_tuner import RandomSearch

In [3]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,mean_absolute_percentage_error

In [4]:
import relative_accuracy as ra
from statistics import mean, stdev

In [5]:
# Load all Inputs and Output Data

# 5 mins (1 step ahead)
Deep_train_5   = np.load("Deep_train_5.npz")['x'] 
Output_train_5 = np.load("Deep_train_5.npz")['y']

Deep_test_5   = np.load("Deep_test_5.npz")['x'] 
Output_test_5 = np.load("Deep_test_5.npz")['y'] 

# 15 mins (3 steps ahead)
Deep_train_15   = np.load("Deep_train_15.npz")['x'] 
Output_train_15 = np.load("Deep_train_15.npz")['y']

Deep_test_15   = np.load("Deep_test_15.npz")['x'] 
Output_test_15 = np.load("Deep_test_15.npz")['y']

# 30 mins (6 steps ahead)
Deep_train_30   = np.load("Deep_train_30.npz")['x'] 
Output_train_30 = np.load("Deep_train_30.npz")['y']

Deep_test_30   = np.load("Deep_test_30.npz")['x'] 
Output_test_30 = np.load("Deep_test_30.npz")['y']

# 60 mins (12 steps ahead)
Deep_train_60   = np.load("Deep_train_60.npz")['x'] 
Output_train_60 = np.load("Deep_train_60.npz")['y']

Deep_test_60   = np.load("Deep_test_60.npz")['x'] 
Output_test_60 = np.load("Deep_test_60.npz")['y']

In [6]:
# Wide Week Data
Xtrain = joblib.load("002weeks_train.save") 
Xtest = joblib.load("002weeks_test.save") 

# Wide Day Data
Xtrain_day = joblib.load("002days_train.save") 
Xtest_day = joblib.load("002days_test.save") 

In [7]:
# 5 mins
# Delete first 15 samples
Wide_train_5 = np.delete(Xtrain, np.s_[0:15], 0)
Wide_test_5 = np.delete(Xtest, np.s_[0:15], 0)

Wide_train_5_day = np.delete(Xtrain_day, np.s_[0:15], 0)
Wide_test_5_day = np.delete(Xtest_day, np.s_[0:15], 0)

In [8]:
# 15 mins
# Delete first 17 samples
Wide_train_15 = np.delete(Xtrain, np.s_[0:17], 0)
Wide_test_15 = np.delete(Xtest, np.s_[0:17], 0)

Wide_train_15_day = np.delete(Xtrain_day, np.s_[0:17], 0)
Wide_test_15_day = np.delete(Xtest_day, np.s_[0:17], 0)

In [9]:
# 30 mins
# Delete first 20 samples
Wide_train_30 = np.delete(Xtrain, np.s_[0:20], 0)
Wide_test_30 = np.delete(Xtest, np.s_[0:20], 0)

Wide_train_30_day = np.delete(Xtrain_day, np.s_[0:20], 0)
Wide_test_30_day = np.delete(Xtest_day, np.s_[0:20], 0)

In [10]:
# 60 mins
# Delete first 27 samples
Wide_train_60 = np.delete(Xtrain, np.s_[0:26], 0)
Wide_test_60 = np.delete(Xtest, np.s_[0:26], 0)

Wide_train_60_day = np.delete(Xtrain_day, np.s_[0:26], 0)
Wide_test_60_day = np.delete(Xtest_day, np.s_[0:26], 0)

In [11]:
scaler_filename = "scaler.save"
scaler = joblib.load(scaler_filename) 

In [12]:
Test      = pd.read_csv('01test_scaled.csv', index_col=None, parse_dates=[0])

# Reshape Samples

The first step is to split the input sequences into subsequences that can be processed by the CNN model. Here, each spatio-temporal sample can be split into three sub-samples, each with five time steps. The CNN can interpret each subsequence of five time steps and provide a time series of interpretations of the subsequences to the LSTM model to process as input.

In [13]:
# reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
n_features = 6              # No of loop detectors
n_seq = 3                    # Subsequences
n_steps = 5                  # time-step per subsequence
val_percent = 0.07567        # 2 weeks
batch_size = 32

#### 5 mins

In [14]:
# Train
# reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
Xt5 = Deep_train_5.reshape((Deep_train_5.shape[0], n_seq, n_steps, n_features))
yt5 = Output_train_5

In [15]:
# Test
# reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
Xv5 = Deep_test_5.reshape((Deep_test_5.shape[0], n_seq, n_steps, n_features))
yv5 = Output_test_5

#### 15 mins

In [16]:
# Train
# reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
Xt15 = Deep_train_15.reshape((Deep_train_15.shape[0], n_seq, n_steps, n_features))
yt15 = Output_train_15

In [17]:
# Test
# reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
Xv15 = Deep_test_15.reshape((Deep_test_15.shape[0], n_seq, n_steps, n_features))
yv15 = Output_test_15

#### 30 mins

In [18]:
# Train
# reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
Xt30 = Deep_train_30.reshape((Deep_train_30.shape[0], n_seq, n_steps, n_features))
yt30 = Output_train_30

In [19]:
# Test
# reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
Xv30 = Deep_test_30.reshape((Deep_test_30.shape[0], n_seq, n_steps, n_features))
yv30 = Output_test_30

#### 60 mins

In [20]:
# Train
# reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
Xt60 = Deep_train_60.reshape((Deep_train_60.shape[0], n_seq, n_steps, n_features))
yt60 = Output_train_60

In [21]:
# Test
# reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
Xv60 = Deep_test_60.reshape((Deep_test_60.shape[0], n_seq, n_steps, n_features))
yv60 = Output_test_60

### Reshaping the Data for Conv2D layer

For Conv2D, there is a need to add one more dimension to show we're dealing with 1 channel (since technically the images are in black and white, only showing values from 0-max flow on a single channel).

Conv1D - strides in 1 dimension
Conv2D - strides in 2 dimensions

In [22]:
# define no_of_channels
n_channels = 1

In [23]:
# Train data
Xt5  =  Xt5.reshape(Xt5.shape[0], n_seq, n_steps, n_features, n_channels)
Xt15 =  Xt15.reshape(Xt15.shape[0], n_seq, n_steps, n_features, n_channels)
Xt30 =  Xt30.reshape(Xt30.shape[0], n_seq, n_steps, n_features, n_channels)
Xt60 =  Xt60.reshape(Xt60.shape[0], n_seq, n_steps, n_features, n_channels)

# Validation data
Xv5  =  Xv5.reshape(Xv5.shape[0], n_seq, n_steps, n_features, n_channels)
Xv15 =  Xv15.reshape(Xv15.shape[0], n_seq, n_steps, n_features, n_channels)
Xv30 =  Xv30.reshape(Xv30.shape[0], n_seq, n_steps, n_features, n_channels)
Xv60 =  Xv60.reshape(Xv60.shape[0], n_seq, n_steps, n_features, n_channels)

In [24]:
Wide_train_5 = np.expand_dims(Wide_train_5, 1)
Wide_train_15 = np.expand_dims(Wide_train_15, 1)
Wide_train_30 = np.expand_dims(Wide_train_30, 1)
Wide_train_60 = np.expand_dims(Wide_train_60, 1)

Wide_test_5 = np.expand_dims(Wide_test_5, 1)
Wide_test_15 = np.expand_dims(Wide_test_15, 1)
Wide_test_30 = np.expand_dims(Wide_test_30, 1)
Wide_test_60 = np.expand_dims(Wide_test_60, 1)

In [25]:
Wide_train_5_day = np.expand_dims(Wide_train_5_day, 1)
Wide_train_15_day = np.expand_dims(Wide_train_15_day, 1)
Wide_train_30_day = np.expand_dims(Wide_train_30_day, 1)
Wide_train_60_day = np.expand_dims(Wide_train_60_day, 1)

Wide_test_5_day = np.expand_dims(Wide_test_5_day, 1)
Wide_test_15_day = np.expand_dims(Wide_test_15_day, 1)
Wide_test_30_day = np.expand_dims(Wide_test_30_day, 1)
Wide_test_60_day = np.expand_dims(Wide_test_60_day, 1)

In [26]:
Wt5  = Wide_train_5
Wt15 = Wide_train_15
Wt30 = Wide_train_30
Wt60 = Wide_train_60

Wv5  = Wide_test_5
Wv15 = Wide_test_15
Wv30 = Wide_test_30
Wv60 = Wide_test_60

In [27]:
Wt5x  = Wide_train_5_day
Wt15x = Wide_train_15_day
Wt30x = Wide_train_30_day
Wt60x = Wide_train_60_day

Wv5x  = Wide_test_5_day
Wv15x = Wide_test_15_day
Wv30x = Wide_test_30_day
Wv60x = Wide_test_60_day

# CNN-LSTM model

In [28]:
from tensorflow import keras
from keras_self_attention import SeqSelfAttention

# Early Stopping
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss',patience=5)

In [29]:
stop = keras.callbacks.EarlyStopping(
    monitor = 'val_loss', 
    patience = 5, 
    restore_best_weights = True)

In [30]:
Wide_train_5.shape

(49233, 1, 14)

In [31]:
Xt5.shape[1:]

(3, 5, 6, 1)

# 30 mins ahead

In [32]:
def build_model_30(hp):
    # Inputs
    channel_Wide = keras.layers.Input(shape=Wide_train_30.shape[1:], name="wide_week")
    channel_Widex = keras.layers.Input(shape=Wide_train_30_day.shape[1:], name="wide_day")
    channel_Deep = keras.layers.Input(shape=Xt30.shape[1:], name="deep_input")
    
    # Wide Model
    Wide_30 = keras.layers.Bidirectional(
        keras.layers.LSTM(units=hp.Int("wide_week_LSTM", min_value=1, max_value=100, step=1), 
                                activation='relu'))(channel_Wide)
    
    Wide_30x = keras.layers.Bidirectional(
        keras.layers.LSTM(units=hp.Int("wide_day_LSTM", min_value=1, max_value=100, step=1), 
                                activation='relu'))(channel_Widex)
    
    # Deep Model
    # CNN 
    CNN_30a = keras.layers.TimeDistributed(
        keras.layers.Conv2D(filters=hp.Int("filters", min_value=32, max_value=512, step=32),
                            kernel_size=hp.Choice("kernel_size", [2, 3]), activation='relu'))(channel_Deep)
    CNN_30b = keras.layers.TimeDistributed(
        keras.layers.Conv2D(filters=hp.Int("filters", min_value=32, max_value=512, step=32),
                            kernel_size=hp.Choice("kernel_size", [2, 3]), activation='relu'))(CNN_30a)
    flatten_30 = keras.layers.TimeDistributed(keras.layers.Flatten())(CNN_30b)
    # LSTM          
    LSTM_30a = keras.layers.LSTM(units=hp.Int("units_LSTM", min_value=1, max_value=100, step=1), 
                                activation='relu', return_sequences=True)(flatten_30)
    LSTM_30b = keras.layers.LSTM(units=hp.Int("units_LSTM", min_value=1, max_value=100, step=1), 
                                activation='relu', return_sequences=True)(LSTM_30a)
    Att_30 = SeqSelfAttention(attention_activation='tanh')(LSTM_30b)
    Reshaped_30 = keras.layers.Flatten()(Att_30)
    
    # Concatenation 
    concat = keras.layers.concatenate([Wide_30, Wide_30x, Reshaped_30])
    
    # Output
    output = keras.layers.Dense(n_features, name= "output")(concat)
    
    # Model
    model_30 = keras.Model(inputs=[channel_Wide, channel_Widex, channel_Deep], outputs=[output])

    # Compile
    model_30.compile(optimizer=keras.optimizers.Adam(hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])), 
        loss='mse',metrics=['MeanAbsoluteError','RootMeanSquaredError','MeanAbsolutePercentageError'])
                    
    return model_30

In [33]:
tuner30 = RandomSearch(
    build_model_30,
    objective="val_loss",
    max_trials=30,
    executions_per_trial=2,
    overwrite=False,
    directory=os.path.normpath('C:/RunsOak'),
    project_name="8a-Conv-LSTM-30",
)

In [34]:
tuner30.search_space_summary()

Search space summary
Default search space size: 6
wide_week_LSTM (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 100, 'step': 1, 'sampling': None}
wide_day_LSTM (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 100, 'step': 1, 'sampling': None}
filters (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
kernel_size (Choice)
{'default': 2, 'conditions': [], 'values': [2, 3], 'ordered': True}
units_LSTM (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 100, 'step': 1, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [35]:
tuner30.search((Wt30, Wt30x, Xt30), yt30, epochs=200,
            validation_split = val_percent,
            callbacks=[early_stop],
            verbose=2)

Trial 30 Complete [00h 16m 22s]
val_loss: 0.002025944762863219

Best val_loss So Far: 0.0020111437188461423
Total elapsed time: 20h 47m 44s
INFO:tensorflow:Oracle triggered exit


In [36]:
tuner30.results_summary(1)

Results summary
Results in C:\RunsOak\8a-Conv-LSTM-30
Showing 1 best trials
Objective(name='val_loss', direction='min')
Trial summary
Hyperparameters:
wide_week_LSTM: 87
wide_day_LSTM: 66
filters: 480
kernel_size: 2
units_LSTM: 75
learning_rate: 0.001
Score: 0.0020111437188461423


### Best model

In [37]:
best_hp30 = tuner30.get_best_hyperparameters()[0]
model30 = tuner30.hypermodel.build(best_hp30)

In [38]:
scaler_filename = "8a-Conv_LSTM-30"
joblib.dump(best_hp30, scaler_filename) 

['8a-Conv_LSTM-30']

In [39]:
model30.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 deep_input (InputLayer)        [(None, 3, 5, 6, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 time_distributed_3 (TimeDistri  (None, 3, 4, 5, 480  2400       ['deep_input[0][0]']             
 buted)                         )                                                                 
                                                                                                  
 time_distributed_4 (TimeDistri  (None, 3, 3, 4, 480  922080     ['time_distributed_3[0][0]']     
 buted)                         )                                                           

# Model 60

In [40]:
def build_model_60(hp):
    # Inputs
    channel_Wide = keras.layers.Input(shape=Wide_train_60.shape[1:], name="wide_week")
    channel_Widex = keras.layers.Input(shape=Wide_train_60_day.shape[1:], name="wide_day")
    channel_Deep = keras.layers.Input(shape=Xt60.shape[1:], name="deep_input")
    
    # Wide Model
    Wide_60 = keras.layers.Bidirectional(
        keras.layers.LSTM(units=hp.Int("wide_week_LSTM", min_value=1, max_value=100, step=1), 
                                activation='relu'))(channel_Wide)
    
    Wide_60x = keras.layers.Bidirectional(
        keras.layers.LSTM(units=hp.Int("wide_day_LSTM", min_value=1, max_value=100, step=1), 
                                activation='relu'))(channel_Widex)
    
    # Deep Model
    # CNN 
    CNN_60a = keras.layers.TimeDistributed(
        keras.layers.Conv2D(filters=hp.Int("filters", min_value=32, max_value=512, step=32),
                            kernel_size=hp.Choice("kernel_size", [2, 3]), activation='relu'))(channel_Deep)
    CNN_60b = keras.layers.TimeDistributed(
        keras.layers.Conv2D(filters=hp.Int("filters", min_value=32, max_value=512, step=32),
                            kernel_size=hp.Choice("kernel_size", [2, 3]), activation='relu'))(CNN_60a)
    flatten_60 = keras.layers.TimeDistributed(keras.layers.Flatten())(CNN_60b)
    # LSTM          
    LSTM_60a = keras.layers.LSTM(units=hp.Int("units_LSTM", min_value=1, max_value=100, step=1), 
                                activation='relu', return_sequences=True)(flatten_60)
    LSTM_60b = keras.layers.LSTM(units=hp.Int("units_LSTM", min_value=1, max_value=100, step=1), 
                                activation='relu', return_sequences=True)(LSTM_60a)
    Att_60 = SeqSelfAttention(attention_activation='tanh')(LSTM_60b)
    Reshaped_60 = keras.layers.Flatten()(Att_60)
    
    # Concatenation 
    concat = keras.layers.concatenate([Wide_60, Wide_60x, Reshaped_60])
    
    # Output
    output = keras.layers.Dense(n_features, name= "output")(concat)
    
    # Model
    model_60 = keras.Model(inputs=[channel_Wide, channel_Widex, channel_Deep], outputs=[output])

    # Compile
    model_60.compile(optimizer=keras.optimizers.Adam(hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])), 
        loss='mse',metrics=['MeanAbsoluteError','RootMeanSquaredError','MeanAbsolutePercentageError'])
                    
    return model_60

In [41]:
tuner60 = RandomSearch(
    build_model_60,
    objective="val_loss",
    max_trials=30,
    executions_per_trial=2,
    overwrite=False,
    directory=os.path.normpath('C:/RunsOak'),
    project_name="8a-Conv-LSTM-60",
)

In [42]:
tuner60.search_space_summary()

Search space summary
Default search space size: 6
wide_week_LSTM (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 100, 'step': 1, 'sampling': None}
wide_day_LSTM (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 100, 'step': 1, 'sampling': None}
filters (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
kernel_size (Choice)
{'default': 2, 'conditions': [], 'values': [2, 3], 'ordered': True}
units_LSTM (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 100, 'step': 1, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [43]:
tuner60.search((Wt60, Wt60x, Xt60), yt60, epochs=200,
            validation_split = val_percent,
            callbacks=[early_stop],
            verbose=2)

Trial 30 Complete [01h 22m 18s]
val_loss: 0.0022131188306957483

Best val_loss So Far: 0.0021462483564391732
Total elapsed time: 01h 13m 22s
INFO:tensorflow:Oracle triggered exit


In [44]:
tuner60.results_summary(1)

Results summary
Results in C:\RunsOak\8a-Conv-LSTM-60
Showing 1 best trials
Objective(name='val_loss', direction='min')
Trial summary
Hyperparameters:
wide_week_LSTM: 77
wide_day_LSTM: 61
filters: 256
kernel_size: 3
units_LSTM: 63
learning_rate: 0.001
Score: 0.0021462483564391732


### Best model

In [45]:
best_hp60 = tuner60.get_best_hyperparameters()[0]
model60 = tuner60.hypermodel.build(best_hp60)

In [46]:
scaler_filename = "Best_HP/8a-Conv_LSTM-60"
joblib.dump(best_hp60, scaler_filename) 

['Best_HP/8a-Conv_LSTM-60']

In [47]:
model60.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 deep_input (InputLayer)        [(None, 3, 5, 6, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 time_distributed_3 (TimeDistri  (None, 3, 3, 4, 256  2560       ['deep_input[0][0]']             
 buted)                         )                                                                 
                                                                                                  
 time_distributed_4 (TimeDistri  (None, 3, 1, 2, 256  590080     ['time_distributed_3[0][0]']     
 buted)                         )                                                           